In [ ]:
import random
from math import inf


params = {
    "farmer_productivity": 3,
    
    
    # "land_capacity": 120,

    "steal_rate": 3,
    "suppression_prob": 0.3,
    "confiscate_rate": 0.7,

    "tax_rate": .5,
    "soldier_base_pay": 2.0,

    "alpha_perception": 0.25,
    "fear_increment": 0.1,
    "fear_decay": 0.2,

    "utility_margin": 0.5,
    "switch_urge_threshold": 2.0,
}

EPS = 1e-6
FEAR_MAX = 10.0


class Person:
    """
    Represents a single agent in the simulation.

    Each person has:
    - An economic role: farmer, bandit, or soldier
    - Economic state: wealth and income flows
    - Psychological state: fear, perceived violence, loyalty
    - Behavioral state: role-switching logic with inertia + cooldown
    - Individual heterogeneity: productivity, aggression, and enforcement skill

    This class handles:
    - Individual actions (produce, rob, suppress, pay tax, receive pay)
    - Psychological updates
    - Forward-looking utility prediction
    - Role switching decisions
    """

    def __init__(self, role, wealth=0.0, prod_mult=1.0, aggr_mult=1.0, skill_mult=1.0):
        """
        Initialize a new agent.

        Parameters:
        - role: Initial role ("farmer", "bandit", or "soldier")
        - wealth: Initial wealth level
        - prod_mult: Productivity multiplier (heterogeneity)
        - aggr_mult: Aggression multiplier (how much they steal)
        - skill_mult: Skill multiplier for suppression
        """
        self.role = role
        self.wealth = max(0.0, wealth)

        # ---心理 / psychological traits ---
        self.fear = 0.0                         # internal fear level
        self.perceived_violence = 0.0           # smoothed estimate of world violence
        self.loyalty = random.uniform(0.8, 1.2) # resistance to switching roles

        # --- Role switching state ---
        self.next_role = role
        self.age_in_role = 0                   # how long they've stayed
        self.switch_cooldown = 3               # inertia to prevent constant switching

        # --- Last tick economics (for logging/diagnostics) ---
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

        # --- Heterogeneity among agents ---
        # These override the passed base values with randomness
        self.prod_mult = random.uniform(0.9, 1.1)
        self.aggr_mult = random.uniform(0.9, 1.1)
        self.skill_mult = random.uniform(0.9, 1.1)

    # ================= Housekeeping ================= #

    def reset_for_new_tick(self):
        """
        Reset per-tick variables and update internal counters.
        Called once at the beginning of every world step.
        """
        self.next_role = self.role
        self.age_in_role += 1

        # cooldown prevents constant oscillating behavior
        if self.switch_cooldown > 0:
            self.switch_cooldown -= 1

        # reset logs
        self.last_production = 0.0
        self.last_rob_gain = 0.0
        self.last_pay = 0.0

    # ================= Role detectors ================= #

    def is_farmer(self):
        """Return True if this agent is a farmer."""
        return self.role == "farmer"

    def is_bandit(self):
        """Return True if this agent is a bandit."""
        return self.role == "bandit"

    def is_soldier(self):
        """Return True if this agent is a soldier."""
        return self.role == "soldier"

    # ================= Role actions ================= #

    def produce(self, farmer_productivity, scale=1.0):
        """
        Farmer produces wealth.

        Production depends on:
        - Fixed productivity parameter
        - Individual productivity multiplier
        - Optional global scale (for resource scarcity, etc.)
        """
        if not self.is_farmer():
            return 0.0

        inc = max(0.0, self.prod_mult * farmer_productivity * max(0.0, scale))
        self.wealth += inc
        self.last_production = inc
        return inc

    def attempt_rob(self, victim, steal_rate, fear_bump_robbed=0.3):
        """
        Bandit attempts to steal from a farmer.

        If successful:
        - Transfers stolen wealth
        - Increases victim fear
        - Logs stolen wealth
        """
        if not (self.is_bandit() and victim and victim.role == "farmer"):
            return 0.0, False

        stolen = min(self.aggr_mult * steal_rate, victim.wealth)

        if stolen <= 0:
            return 0.0, False

        victim.wealth -= stolen
        victim.fear = min(FEAR_MAX, victim.fear + fear_bump_robbed)

        self.wealth += stolen
        self.last_rob_gain = stolen

        return stolen, True

    def attempt_suppress(self, target_bandit, p_hit, confiscate_rate, fear_bump_suppressed=0.2):
        """
        Soldier attempts to suppress a bandit.

        If successful:
        - Confiscates stolen wealth
        - Raises bandit's fear
        - Gains confiscated wealth
        """

        if not (self.is_soldier() and target_bandit and target_bandit.role == "bandit"):
            return 0.0, False

        hit_prob = min(1.0, max(0.0, self.skill_mult * p_hit))

        if random.random() >= hit_prob:
            return 0.0, False

        amt = min(self.skill_mult * confiscate_rate, target_bandit.wealth)

        target_bandit.wealth -= amt
        target_bandit.fear = min(FEAR_MAX, target_bandit.fear + fear_bump_suppressed)

        self.wealth += amt
        return amt, True

    def pay_tax(self, tax_rate):
        """
        Farmer pays tax as a percentage of wealth.
        Funds are redistributed to soldiers.
        """
        if not self.is_farmer():
            return 0.0

        tax = tax_rate * self.wealth
        self.wealth -= tax

        return tax

    def receive_pay(self, amount):
        """
        Soldier receives salary from tax pool + government base pay.
        """
        if amount > 0:
            self.wealth += amount
            self.last_pay = amount

    # ================= Psychological updates ================= #

    def update_psychology(self, global_violence, alpha, fear_inc, fear_dec):
        """
        Update internal psychological state:
        - Update perceived violence using exponential smoothing
        - Translate perceived violence into fear changes
        """
        a = max(0.0, min(1.0, alpha))

        self.perceived_violence = (1 - a) * self.perceived_violence + a * global_violence

        if self.perceived_violence > 0.5:
            self.fear = min(FEAR_MAX, self.fear + fear_inc)
        else:
            self.fear = max(0.0, self.fear - fear_dec)

    # ================= Forward-looking utility ================= #

    def predicted_utility(self, role, world, params):
        """
        Predict utility of choosing a given role NEXT tick.

        This is a forward-looking expectation based on:
        - Population composition
        - Global violence
        - Tax rates, productivity, and suppression parameters
        """

        # Hypothetical population counts if this person switched
        f, b, s = world.farmer_pop, world.bandit_pop, world.soldier_pop

        if self.is_farmer(): f -= 1
        elif self.is_bandit(): b -= 1
        elif self.is_soldier(): s -= 1

        if role == "farmer": f += 1
        elif role == "bandit": b += 1
        elif role == "soldier": s += 1

        violence = world.total_violence_last_tick

        # --- Farmer utility ---
        if role == "farmer":
            predicted_prod = self.prod_mult * params["farmer_productivity"]
            safety_bonus = max(0.0, 0.5 - 0.02 * violence)
            return predicted_prod + safety_bonus - 0.01 * b

        # --- Bandit utility ---
        if role == "bandit":
            if f > 0:
                base_rob = self.aggr_mult * params["steal_rate"]
            else:
                base_rob = 0.0
            return (
                base_rob
                - 0.01 * b                     # competition
                + 0.05 * max(0, f)        # more farmers => more loot
                - 0.02 * s                     # soldiers make life worse
            )

        # --- Soldier utility ---
        if role == "soldier":
            tax_pool = f * params["farmer_productivity"] * params["tax_rate"] if f > 0 else 0.0
            base_total = params["soldier_base_pay"] * s
            pay_per = (tax_pool + base_total) / s if s > 0 else params["soldier_base_pay"]

            return pay_per + 0.02 * b - 0.30

    # ================= Switching logic ================= #

    def decide_switch(self, utilities, switch_urge_threshold, utility_margin, global_tick):
        """
        Decide whether to switch roles.

        Switching depends on:
        - Fear (creates switching urge)
        - Loyalty (resistance factor)
        - Utility comparison
        - Cooldown and startup freeze
        """

        if global_tick <= 5:
            self.next_role = self.role
            return

        if self.switch_cooldown > 0:
            self.next_role = self.role
            return

        urge = self.fear / (self.loyalty + EPS)
        current_u = utilities[self.role]

        if urge <= switch_urge_threshold:
            self.next_role = self.role
            return

        # find best role by utility
        best_role = max(utilities, key=lambda r: utilities[r])

        if utilities[best_role] > current_u + utility_margin:
            self.next_role = best_role

    def finalize_role_switch(self):
        """
        Commit the switch decided earlier and reset role age + cooldown.
        """
        if self.next_role != self.role:
            self.role = self.next_role
            self.age_in_role = 0
            self.switch_cooldown = 1

    # ================= Export ================= #

    def to_dict(self):
        """
        Export agent data for logging, visualization, or analysis.
        """
        return {
            "role": self.role,
            "wealth": round(self.wealth, 3),
            "fear": round(self.fear, 3),
            "perceived_violence": round(self.perceived_violence, 3),
            "loyalty": round(self.loyalty, 3),
            "age_in_role": self.age_in_role,
        }

class World:
    """
    Represents the simulation environment.

    The World handles:
    - Population tracking
    - Running each simulation tick
    - Aggregating violence and economics
    - Updating global utilities and triggering role transitions
    """

    def __init__(self, n_farmers, n_bandits, n_soilders):
        """
        Initialize world with given population counts.
        """
        self.people = []
        self.global_tick = 0

        for _ in range(n_farmers):
            self.people.append(Person("farmer", wealth=1.0))
        for _ in range(n_bandits):
            self.people.append(Person("bandit", wealth=0.7))
        for _ in range(n_soilders):
            self.people.append(Person("soldier", wealth=0.8))

        self.total_robberies = 0
        self.total_suppressions = 0
        self.total_violence_last_tick = 0.0

        self.utilities_for_roles = {"farmer": 0.0, "bandit": 0.0, "soldier": 0.0}

        self.update_population_counts()

        # Diagnostics
        self.expected_prod_per_farmer = 0.0
        self.expected_rob_per_bandit = 0.0
        self.expected_soldier_pay = 0.0

    def update_population_counts(self):
        """
        Recalculate number of farmers, bandits, and soldiers.
        Should be called after role switches.
        """
        self.farmer_pop = sum(p.is_farmer() for p in self.people)
        self.bandit_pop = sum(p.is_bandit() for p in self.people)
        self.soldier_pop = sum(p.is_soldier() for p in self.people)

    def step(self):
        """
        Advance the simulation by one tick.

        This function performs:
        1. Production
        2. Robbery attempts
        3. Suppression by soldiers
        4. Tax redistribution
        5. Psychology updates
        6. Forward utility calculation
        7. Role switching
        """
        self.global_tick += 1

        # Reset all agents for new cycle
        for p in self.people:
            p.reset_for_new_tick()

        farmers = [p for p in self.people if p.is_farmer()]
        bandits = [p for p in self.people if p.is_bandit()]
        soldiers = [p for p in self.people if p.is_soldier()]

        # --- Production phase ---
        for f in farmers:
            f.produce(params["farmer_productivity"])

        # --- Robbery phase ---
        self.total_robberies = 0
        random.shuffle(bandits)

        for b in bandits:
            if not farmers:
                break
            target = max(farmers, key=lambda x: x.wealth)

            _, success = b.attempt_rob(target, params["steal_rate"])
            if success:
                self.total_robberies += 1

        # --- Suppression phase ---
        self.total_suppressions = 0
        bandits = [p for p in self.people if p.is_bandit()]

        for s in soldiers:
            if not bandits:
                break

            target = random.choice(bandits)

            _, success = s.attempt_suppress(
                target_bandit=target,
                p_hit=params["suppression_prob"],
                confiscate_rate=params["confiscate_rate"],
            )

            if success:
                self.total_suppressions += 1

        # --- Taxation + soldier pay ---
        tax_pool = sum(f.pay_tax(params["tax_rate"]) for f in farmers)
        total_base = params["soldier_base_pay"] * len(soldiers)

        pay_per = (tax_pool + total_base) / len(soldiers) if soldiers else 0.0

        for s in soldiers:
            s.receive_pay(pay_per)

        # --- Diagnostics (expected values) ---
        self.expected_prod_per_farmer = (
            sum(f.last_production for f in farmers) / len(farmers) if farmers else 0.0
        )
        self.expected_rob_per_bandit = (
            sum(b.last_rob_gain for b in bandits) / len(bandits) if bandits else 0.0
        )
        self.expected_soldier_pay = (
            sum(s.last_pay for s in soldiers) / len(soldiers) if soldiers else 0.0
        )

        # --- Violence index ---
        current_violence = self.total_robberies + self.total_suppressions
        self.total_violence_last_tick = current_violence

        # --- Forward utilities ---
        n = len(self.people)

        farmer_u = sum(p.predicted_utility("farmer", self, params) for p in self.people) / n
        bandit_u = sum(p.predicted_utility("bandit", self, params) for p in self.people) / n
        soldier_u = sum(p.predicted_utility("soldier", self, params) for p in self.people) / n

        # Hard lower bounds to prevent collapse artifacts
        self.utilities_for_roles = {
            "farmer": max(1.5, max(farmer_u, self.expected_prod_per_farmer)),
            "bandit": max(1.5, max(bandit_u, self.expected_rob_per_bandit)),
            "soldier": max(1.5, max(soldier_u, self.expected_soldier_pay) - 0.5)
        }

        # --- Update internal psychology ---
        for p in self.people:
            p.update_psychology(
                global_violence=current_violence,
                alpha=params["alpha_perception"],
                fear_inc=params["fear_increment"],
                fear_dec=params["fear_decay"],
            )

        # --- Role switching ---
        for p in self.people:
            p.decide_switch(
                utilities=self.utilities_for_roles,
                switch_urge_threshold=params["switch_urge_threshold"],
                utility_margin=random.uniform(0.1, 2),
                global_tick=self.global_tick,
            )

        for p in self.people:
            p.finalize_role_switch()

        self.update_population_counts()

    def to_dict(self):
        """
        Export global state for visualization or CSV logging.
        """
        return {
            "farmer_pop": self.farmer_pop,
            "bandit_pop": self.bandit_pop,
            "soldier_pop": self.soldier_pop,
            "total_robberies": self.total_robberies,
            "total_suppressions": self.total_suppressions,
            "utilities": self.utilities_for_roles,
            "exp_prod_farmer": self.expected_prod_per_farmer,
            "exp_rob_bandit": self.expected_rob_per_bandit,
            "exp_pay_soldier": self.expected_soldier_pay,
        }


In [ ]:
farmer_pop = []
bandit_pop = []
soldier_pop = []

farmer_ut = []
bandit_ut = []
soldier_ut = []

In [ ]:
if __name__ == "__main__":
    world = World(n_farmers=100, n_bandits=100, n_soilders=100)

    for t in range(1, 101):
        world.step()
        state = world.to_dict()

        farmer_pop.append(world.farmer_pop)
        bandit_pop.append(world.bandit_pop)
        soldier_pop.append(world.soldier_pop)

        farmer_ut.append(state['utilities']['farmer'])
        bandit_ut.append(state['utilities']['bandit'])
        soldier_ut.append(state['utilities']['soldier'])
        
        print(
            f"t={t:03d} | F={state['farmer_pop']:3d} "
            f"B={state['bandit_pop']:3d} S={state['soldier_pop']:3d} | "
            f"Robs={state['total_robberies']:2d} Supp={state['total_suppressions']:2d} | "
            f"U_F={state['utilities']['farmer']:.2f} "
            f"U_B={state['utilities']['bandit']:.2f} "
            f"U_S={state['utilities']['soldier']:.2f}"
        )

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame([farmer_pop, soldier_pop, bandit_pop])
df2 = pd.DataFrame([farmer_ut, soldier_ut, bandit_ut])

In [ ]:
df = df.T.reset_index()
df2 = df2.T.reset_index()

In [ ]:
plt.plot(df['index'], df[0], c='r')
plt.plot(df['index'], df[1], c='b')
plt.plot(df['index'], df[2], c='g')
plt.title('Population vs. Iteration')
plt.ylabel('Population')
plt.xlabel('Iteration')
plt.legend(['farmers', 'soldiers', 'bandits'])

In [ ]:
plt.plot(df2['index'], df2[0], c='r')
plt.plot(df2['index'], df2[1], c='b')
plt.plot(df2['index'], df2[2], c='g')
plt.title('Utility vs. Iteration')
plt.ylabel('Utility')
plt.xlabel('Iteration')
plt.ylim(0, 20)
plt.legend(['farmers', 'soldiers', 'bandits'])